# Geerdete Schachtel mit Potentialverteilung auf Deckel

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook wird die Laplace-Gleichung innerhalb eines an den Seiten und auf dem Boden geerdeten Quaders gelöst. Auf dem Deckel des Quaders wird die Potentialverteilung vorgegeben. Zur Bestimmung des Potentials im Innern des Quaders werden Lösungen der Laplace-Gleichung verwendet, die an die Dirichlet-Randbedingungen auf den geerdeten Rändern angepasst sind. Diese Lösungen sind durch

$$\Phi_{m, n}(x, y, z) = \sin(m\pi\frac{x}{L})\sin(n\pi\frac{y}{L})\sinh(\sqrt{m^2+n^2}\pi\frac{z}{L})$$

gegeben. Die Entwicklungskoeffizienten ergeben sich durch eine zweidimensionale Fouriertransformation des Potentials auf dem Deckel bei $z=h$.

## Importanweisungen

In [ ]:
import numpy as np
from scipy import fft
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

## Zwei Deckelpotentiale

Im Folgenden werden zwei Potentialverteilungen auf dem Deckel definiert:
- `constant_potential`: Das dimensionslose Potential wird auf dem gesamten Deckel gleich eins gesetzt.
- `two_disk_potential`: Das dimensionslose Potential wird in zwei Scheiben mit Radius $L/8$ an den Stellen $x=L/4, y=L/2$ und $x=3L/4, y=L/2$ gleich eins gesetzt. Ansonsten ist das Potential gleich null.

Bei Bedarf können weitere Potentialverteilungen definiert werden. Als Argument muss die Zahl `n` der Gitterpunkte je Dimension eines Quadratgitters übergeben werden. Zurückgegeben werden muss ein NumPy-Array der Form `(n, n)`, das die Potentialwerte auf dem Gitter enthält.

In [ ]:
def constant_potential(n):
    return np.ones((n, n), dtype=float)

def two_disk_potential(n):
    potential = np.zeros((n, n), dtype=float)
    xidx, yidx = np.ogrid[:n, :n]
    y0 = n/2
    r = n/8
    for x0 in (n/4, 3*n/4):
        disk = (xidx-x0)**2 + (yidx-y0)**2 <= r**2
        potential[disk] = 1
    return potential

## Potential im Quaderinnern

Um das Potential im Quader zu bestimmen, wird zunächst das Deckelpotential auf einem Gitter der Form `(nx, nx)` berechnet und anschließend fouriertransformiert. Die Funktion zur Berechnung des Deckelpotentials wird im Argument `potential_cover` übergeben. Das Argument `h_over_l` gibt das Verhältnis von Quaderhöhe zu Seitenlänge an. Nachdem die Fouriertransformierte mit der benötigten $z$-Abhängigkeit aus den Lösungen $\Phi_{m,n}$ multipliziert wurde, kann die Fourierrücktransformation erfolgen, um das Potential auf einem zweidimensionalen vertikalen Schnitt bei $y=L/2$ in einem Array der Form `(nx, nz)` zu erhalten.

In [ ]:
def potential(potential_cover, h_over_l, nx, nz):
    v = potential_cover(nx)
    v_tilde = fft.dstn(v)
    coeffs = v_tilde[:, :, np.newaxis]
    xidx, yidx, zidx = np.ogrid[1:nx+1, 1:nx+1,
                                0:h_over_l:nz*1j]
    arg1 = np.sqrt(xidx**2 + yidx**2)*np.pi
    coeffs = (coeffs*np.exp(arg1*(zidx-h_over_l))
              * (1-np.exp(-2*arg1*zidx))
              / (1-np.exp(-2*arg1*h_over_l)))
    phi = fft.idstn(coeffs, axes=(0, 1))
    return phi[:, nx//2, :]

## Implementierung der Bedienelemente und graphische Darstellung des elektrischen Potentials

Mit Hilfe der Bedienelemente lassen sich die folgenden Parameter einstellen:
- `v`: Funktion zur Berechnung des Potentials. Es kann zwischen einem konstanten Potential und einem Zweischeibenpotential gewählt werden.
- `h_over_l`: Verhältnis von Quaderhöhe zu Seitenlänge
- `log_nx`: Zweierlogarithmus der Anzahl der Gitterpunkte für $x$- und $y$-Richtung
- `nz`: Zahl der zu betrachtenden Punkte in $z$-Richtung

Das Potential wird in der $x$-$z$-Ebene bei $y=L/2$ farblich dargestellt. Zur besseren Interpretierbarkeit werden noch weiße Äquipotentiallinien eingezeichnet. Die Höhe der Farbskala passt sich der Quaderhöhe an. 

In [ ]:
v_options = [("konstantes Potential", constant_potential),
             ("Zweischeibenpotential", two_disk_potential)]
widget_dict = {"v":
               widgets.widgets.Dropdown(
                   options=v_options,
                   description=r"$\Phi_\text{D}(x,y)$"),
               "h_over_l":
               widgets.FloatSlider(
                   value=1, min=0.1, max=1, step=0.05,
                   description="$h/L$"),
               "log_nx":
               widgets.IntSlider(
                   value=7, min=5, max=9,
                   description=r"$\log_2(n_x)$"),
               "nz":
               widgets.IntSlider(
                   value=200, min=50, max=1000, step=50,
                   description="$n_z$")
               }

@interact(**widget_dict)
def plot_result(v, h_over_l, log_nx, nz):
    nx = 2**log_nx
    phi = potential(v, h_over_l, nx, nz)

    fig, ax = plt.subplots(figsize=(8, 8))
    mesh = ax.imshow(phi.T, origin="lower", cmap="plasma",
                     extent=(0, 1, 0, h_over_l))
    ax.contour(phi.T,
               levels=[0.05+n*0.1 for n in range(10)],
               colors="white", origin="lower",
               extent=(0, 1, 0, h_over_l))
    ax.set_xlabel("$x/L$")
    ax.set_ylabel("$z/L$")
    fig.colorbar(mesh, fraction=0.05*h_over_l)